<center> <h1>Team name: Turquoise</h1> </center>

<img src="images/k.jpg" alt="drawing" width="800"/>


- Members:
	- **Rabia Yasa Kostas**
		- e-mail: rabia.yasakostas@ed.ac.uk
		- phone: +44 7427 011498
	- **Kahraman Kostas**
		- e-mail: kk97@hw.ac.uk
		- phone: +44 7401 541751

The team presenter: **Kahraman Kostas**

<center> <h1>TASK 1</h1> </center> 

**NOTE: it is sufficient to run all cells from top to bottom, taking into account the bolded notes.**

The steps in the first task can be summarized as follows.
- Input files are read, using those files :
    - Training data feature extraction is performed.
    - Testing data feature extraction is performed.
- At the end of feature extraction, two new dataset with 11 features is created (training data contains real distance, testing data does not).
- An artificial neural network (ANN) is created and trained with the Training data.
- Using the trained model, the test data is labelled and the submission file is created.

These steps are illustrated in the image below.

<img src="images/task-1.png" alt="drawing" width="1000"/>

## 1.1 Installing modules
We used [Python 3.6.5](https://www.python.org/downloads/) to create the application file. We included some additional modules that were not included in the example file given at the start of the competition. These modules can be listed as:


| Molules | Task |
| ------ | ------ |
|[ tensorflow ](https://www.tensorflow.org/)| Deep Learning|
|[ Pandas  ](https://pandas.pydata.org/pandas-docs/stable/install.html)|  Data Analysis|
|[ SciPy ](https://scipy.org/) |Distance Computing|



We started with the installation of these modules as the first step.

In [1]:
## 1.1 Installing modules
!pip install tensorflow==2.6.2
!pip install scipy
!pip install pandas

## 1.2  Setting Random Seeds
In this step, we fixed the related random seed to be used in order to obtain repeatable results. In this way, we have provided a deterministic path where we get the same result in every run. However, according to our observations, the results obtained with different computers may differ slightly (±1%)

In [2]:
## 1.2  Setting Random Seeds
seed_value=0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)

import tensorflow as tf

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)

## 1.3 Loading the data

In this section, we load the data that we will use. We took the code and explanations from the sample file given (`Task1-IPS-Challenge-2021.ipynb`).

The `task1_fingerprints.json` contains all the fingerprint information for the problem. That is each entry represents a real scan of the WiFi emitters in an area of the mall. You will find that the same MAC addresses will be present in many of the fingerprints.

The `task1_train.csv` contains the valid training pairs to help you design/train your algorithm. Each `id1-id2` pair has a labelled ground truth distance (in metres) and each id corresponds to a fingerprints from `task1_fingerprints.json`.

The `task1_test.csv` is the same format as `task1_train.csv` but doesn't have the displacements included. 


**Please make sure there are `task1_train.csv`, `task1_test.csv`,`task1_fingerprints.json` files in the `for_contestants` folder!**

In [5]:
# unzip required files
import zipfile
with zipfile.ZipFile("for_contestants.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [3]:
## 1.3 Loading the data
import csv
import json
import os
from tqdm import tqdm

path_to_data = "for_contestants"

with open(os.path.join(path_to_data,"task1_fingerprints.json")) as f:
    fps = json.load(f)
    
with open(os.path.join(path_to_data,"task1_train.csv")) as f:
    train_data = []
    train_h = csv.DictReader(f)
    for pair in tqdm(train_h):
        train_data.append([pair['id1'],pair['id2'],float(pair['displacement'])])
        
with open(os.path.join(path_to_data,"task1_test.csv")) as f:
    test_h = csv.DictReader(f)
    test_ids = []
    for pair in tqdm(test_h):
        test_ids.append([pair['id1'],pair['id2']])

879824it [00:43, 20404.39it/s] 
5160445it [02:21, 36491.30it/s] 


## 1.4 Feature Extraction

In this step, we perform feature extraction using two functions. `feature_extraction_file` function simply pulls the relevant values of the fingerprints (in pairs) from the JSON file and sends them to the `feature_extraction`  function to do the calculations .


In the `feature_extraction` function, if these two fingerprints are different from each other in terms of size and the devices they contain, all the devices included in the two fingerprints are brought together to form a common sequence without repeating. In each array, we make these two arrays identical (in terms of devices they include) by assigning the value 0 to the non-corresponding devices. This process is explained with an example in the following image.

<img src="images/fp.png" alt="The process of making two different fingerprints similar" width="600"/>

The distance between these two fingerprints, which are made similar, is calculated using 11 different methods\[1\]. These methods are:

- [Bray-Curtis distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.braycurtis.html)
- [Canberra distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.canberra.html)
- [Chebyshev distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.chebyshev.html)
- [City Block (Manhattan) distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cityblock.html)
- [Correlation distance ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.correlation.html)
- [Cosine distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cosine.html)
- [Euclidean distance  ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.euclidean.html)
- [Jensen-Shannon distance ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html)
- [Minkowski distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.minkowski.html)
- [Squared Euclidean distance](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.sqeuclidean.html)
- [Weighted Minkowski distance ](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.wminkowski.html)


Then, these values are directed to the `feature_extraction_file` function and saved as a CSV file within this function. In other words, fingerprints of various sizes turn into an 11-feature CSV file as a result of this process. The model to be used is trained and tested with these newly created features.



In [4]:
## 1.4 Feature Extraction
import scipy.spatial
def feature_extraction_file(data,name,flag):
    features = [["braycurtis",
    "canberra",
    "chebyshev",
    "cityblock",
    "correlation",
    "cosine",
    "euclidean",
    "jensenshannon",
    "minkowski",
    "sqeuclidean",
    "wminkowski", "real"]]
    for i in tqdm((data), position=0, leave=True):
        fp1 = fps[i[0]]
        fp2 = fps[i[1]]
        feature=feature_extraction(fp1,fp2) 
        if flag:
            feature.append(i[2])
        else:feature.append(0)
        features.append(feature)
    with open(name, "w", newline='') as f:
        writer = csv.writer(f)
        writer.writerows(features) 
    #print(features)

In [5]:
## 1.4 Feature Extraction
def feature_extraction(fp1,fp2):
    mac=set(list(fp1.keys())+list(fp2.keys()))
    mac = { i : 0 for i in mac }
    f1=mac.copy()
    f2=mac.copy()
    for key in fp1:
        f1[key]=fp1[key]
    for key in fp2:
        f2[key]=fp2[key]        
    
    f1=list(f1.values())
    f2=list(f2.values())


    
    braycurtis=scipy.spatial.distance.braycurtis(f1,f2)
    canberra=scipy.spatial.distance.canberra(f1,f2)
    chebyshev=scipy.spatial.distance.chebyshev(f1,f2)
    cityblock=scipy.spatial.distance.cityblock(f1,f2)
    correlation=scipy.spatial.distance.correlation(f1,f2)
    cosine=scipy.spatial.distance.cosine(f1,f2)
    euclidean=scipy.spatial.distance.euclidean(f1,f2)
    jensenshannon=scipy.spatial.distance.jensenshannon(f1,f2)
    minkowski=scipy.spatial.distance.minkowski(f1,f2)
    sqeuclidean=scipy.spatial.distance.sqeuclidean(f1,f2)
    wminkowski=scipy.spatial.distance.wminkowski(f1,f2,1, np.ones(len(f1)))



    output_data=[braycurtis,
        canberra,
        chebyshev,
        cityblock,
        correlation,
        cosine,
        euclidean,
        jensenshannon,
        minkowski,
        sqeuclidean,
        wminkowski]
    output_data = [0 if x != x else x for x in output_data]
    return output_data

## 1.5 Model 
In this task, there is fingerprints scans which have RRSI signals from WiFi emitters surroundings in the mall. First challange wants us to estimate the distance between two fingerprints scans which is a regression task. We used ANN (Artificial Neural Networks) which is inspired by biological neural network. ANN consists of three layers; input layer, hidden layers (more than one) and output layer. ANN starts with input layer which includes the training data (with features), passes the data to the first hidden layer where the data is computed by the first hidden layer's weights. In hidden layers, there is an iteration of calculation of weights to the inputs and then apply them an activation function \[2\]. As our problem is regression, our last layer is a single output neuron: its output is the predicted the distances between pairs of fingerprint scans. Our first hidden layer has 64 and the second has 128 neurons. The all architecture of this model is shared as follows.
<img src="images/model.png" alt="The process of making two different fingerprints similar" width="400"/>


We perform  deep learning using two functions.The `create_model` function shapes the training data to train the model and determines the structure of the model. The  `model_features` function produces a model with the specified structure. The created model is saved to be used after being trained by the `create_model` function.



In [6]:
## 1.5 Model 
import scipy.spatial
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from keras.utils.vis_utils import plot_model
%matplotlib inline


def model_features(i,ii):
    model = Sequential()
    model.add(Dense(i, input_shape=(11, ), activation='relu', name='dense_1'))
    model.add(Dense(ii, activation='relu', name='dense_2'))
    model.add(Dense(1, activation='linear', name='dense_output'))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    model.summary()
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    #print(model.get_config())
    return model

def create_model (name):     
    df = pd.read_csv(name)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df=df.fillna(0)
    X =df[df.columns[0:-1]]
    X_train=np.array(X)
    y_train=np.array(df[df.columns[-1]])

    model=model_features(64,128)
    history = model.fit(X_train, y_train, epochs=19, validation_split=0.5)#,batch_size=1)
    loss=history.history['loss']
    val_loss=history.history['val_loss']    


    my_xticks=list(range(len(loss)))
    plt.figure(figsize=(20,5))
    plt.plot(my_xticks,loss, linestyle='-', marker='o', color='b',label= "train")
    plt.plot(my_xticks,val_loss, linestyle='-', marker='o', color='r',label= "val")
    plt.title("Scores ")
    plt.legend(numpoints=1)
    plt.ylabel("Loss")
    plt.xlabel("Epoch")
    plt.xticks(rotation=90) 
    plt.ylim([100, 150]) 
    plt.show()
    madelname="./THEMODEL"
    model.save(madelname)
    print("Model Created!")
    


## 1.6 Checking the inputs

This function checks if the training and testing data have gone through feature extraction. If they have not, it creates these files and the model by calling the corresponding functions. After handling the model and all feature extraction, it formats the test data to produce the final results.

In [7]:
## 1.6 Checking the inputs
from numpy import inf
from numpy import nan

def create_new_files(train,test):
    model_path="./THEMODEL/"
    my_train_file='new_train_features.csv'
    my_test_file='new_test_features.csv'
    
    
    
    if os.path.isfile(my_train_file) :
        pass  
    else:

        print("Please wait! Training data feature extraction is in progress...\nit will take about 10 minutes")
        feature_extraction_file(train,my_train_file,1)
        print("TThe training feature extraction completed!!!")       

        

    if os.path.isfile(my_test_file) :
        pass 
    else:

        print("Please wait! Testing data feature extraction is in progress...\nit will take about 100-120 minutes")
        feature_extraction_file(test,my_test_file,0)
        print("The testing feature extraction completed!!!")           
    

    if  os.path.isdir(model_path):
        pass  
    else:

        print("Please wait! Creating the deep learning model...\nit will take about 10 minutes")
        create_model(my_train_file)
        print("The model file created!!!\n\n\n")   
    
    model = keras.models.load_model(model_path)
    df = pd.read_csv(my_test_file)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df=df.fillna(0)
    X_train =df[df.columns[0:-1]]
    X_train=np.array(X_train)
    y_train=np.array(df[df.columns[-1]])
    predicted=model.predict(X_train)  
    print("Please wait! Creating resuşts... ")
      
    return predicted

## 1.7 Submission 
This step triggers feature extraction and model creation processes and allows all processes to begin. So, using the IDs from the `test1_test.csv` file it fills the third (displacement) column with the estimated distance for this fingerprint pairs and it saves this file in the directory with the name `TASK1-MySubmission.csv`. 

In [11]:
## 1.7 Submission 
distance_estimate = create_new_files(train_data,test_ids)
count=0

output_data = [["id1", "id2", "displacement"]]
for id1, id2 in tqdm(test_ids):   
    output_data.append([id1,id2,distance_estimate[count][0]])
    count+=1
print("Process finished. Preparing result file ...")  
with open("TASK1-MySubmission.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(output_data)
print("The results are ready.\n See MySubmission.csv")  

  2%|█                                                                     | 81482/5160445 [00:00<00:06, 808905.81it/s]

Please wait! Creating resuşts... 


100%|████████████████████████████████████████████████████████████████████| 5160445/5160445 [00:09<00:00, 525830.94it/s]


Process finished. Preparing result file ...
The results are ready.
 See MySubmission.csv


-----------------------

# References

\[1\] P. Virtanen  and SciPy 1.0 Contributors. SciPy 1.0: Fundamental Algorithms for Scientific Computing in Python. Nature Methods, 17:261--272, 2020.

\[2\] A. Geron, Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems. O’Reilly Media, 2019